<a href="https://colab.research.google.com/github/Srijato2005/repo-exercise/blob/main/ytbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [66]:
!pip install -q youtube-transcript-api   langchain_community  langchain_google_genai

In [67]:
from youtube_transcript_api import YouTubeTranscriptApi,TranscriptsDisabled
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate




INDEXING


In [49]:
video_id="FWAdfuPpLOc"
try:
  transcript_list= YouTubeTranscriptApi.get_transcript(video_id,languages=["en"])
  transcipt= " ".join(chunk["text"] for chunk in transcript_list)
  print(transcipt)
except TranscriptsDisabled:
  print("Transcript is disabled for this video")

Who's faster? A cheetah or one of
the fastest cars in the world? Can a man beat a bear in
a fight? Can a tiger beat the world-record
high jumper? We're gonna answer all of these
and more— starting with the strongest man
in the world. And he's about to test his strength
against a 500-pound lion in a game of
tug-of-war. Any last words? It's go time. 3, 2, 1… Go! Alright and
we’re off. Oh my gosh!
Look at his mouth! Look at that
brute strength. Oh my gosh. The lion’s literally holding
the strongest man in the world back right now! Wait, what is he doing? Did that lion just
turn around? It looks like he's trying to
wear Brian out. Here's Brian pulling
a literal plane. But right now, he cannot get
this lion to budge. He’s starting
to struggle! Brian's actually
doing this! Oh, ok. Ok. You got like
3 feet left. You're almost
there! Brian, you’re almost
there! Brian won! Brian won, congrats! Yes! Brian just won! Did you have fun? Oh, ok.
I just— I won't talk to him.
That's fine. But that's jus

In [68]:
splitter= RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks=splitter.create_documents([transcipt])
# chunks[500]
print(f"Number of chunks created: {len(chunks)}")

Number of chunks created: 26


In [69]:
embedding= GoogleGenerativeAIEmbeddings(model='models/embedding-001')
vector_store=FAISS.from_documents(chunks,embedding)
vector_store.index_to_docstore_id

{0: 'd38e813e-027c-424e-a271-aa1184325cc7',
 1: '9794e30f-5e41-4916-96c4-e04c5fbb4465',
 2: '6a3eb4f0-3af2-406f-8b6a-7652b84c9cc2',
 3: '83265d15-0064-4ba3-9630-929579a3ab94',
 4: '12a29803-e6d9-4e70-918a-cfb737db5c66',
 5: '65a4dfa3-fe26-44bf-95c8-53afa7f6631d',
 6: 'ab0ffb5f-02d8-41e4-af8a-8c0dd35d07e9',
 7: 'b3a77f86-2f4e-47a1-9465-b1691db85871',
 8: '2683ccc0-33df-468d-a4ae-5a8b38525596',
 9: 'a7f017c9-0510-4a19-86f0-911d948d59b8',
 10: '784af425-cd52-4994-9f7c-aa7fc3e84be7',
 11: 'e2a84827-7455-42a9-8929-81d12d8cb4ba',
 12: '06d0d24a-65fc-4378-a64a-c24562ef8697',
 13: '59f96873-ea85-4547-9438-59e5644f6ce0',
 14: 'a10ff440-35fc-4918-aab2-3de16c6cfda3',
 15: 'b96ec069-9e1c-40d7-b41a-2ac21edd226d',
 16: '1851bdb0-5faf-49bf-9417-19857b40f71d',
 17: '8b858516-1bcc-4d3b-a0bd-c57c13d1f0d0',
 18: '0dfb3b35-24a1-4351-b34c-32e0b4b3c23d',
 19: '44192e82-cf02-4950-b72b-a83d5cac7165',
 20: 'a3056fd6-ffca-4af0-800b-21e90a21994f',
 21: '1843f29a-6b0c-4cae-ae69-d6dcc0ad8007',
 22: '17589245-952d-

In [52]:
pip install faiss-cpu

In [70]:
retriever =vector_store.as_retriever(search_type="similarity",search_kwargs={"k":4})


In [73]:
retriever.invoke("Does the cheetah overcome the car?")

[Document(id='1843f29a-6b0c-4cae-ae69-d6dcc0ad8007', metadata={}, page_content="so intense, that humans need the speed of\na Formula E race car to stand a chance. Which is exactly why I brought not\njust 1 but 3 cars to try and beat it. All starting with\nthis golf cart vs. cheetah. Oh, look at that thing. Ready boys? - Let's do this.\n- Yup! 3, 2, 1… Send the cheetah! Oh my God,\nit's so fast! Oh, it's not even\nclose. It's crushing us. We’re getting left\nin the dust! I can't even see it anymore,\nit's gone. The cheetah literally\nalready finished, and we're still halfway\nthrough the race. This is embarrassing. Sir! Cheetah, you didn't even give us\na fighting chance. We need something faster.\nBring out the next car! We all knew a golf cart wasn't going\nto beat this cheetah. But for this next race,\nI spared no expense in acquiring one of the most\nbeautiful cars in automotive history. Minivan vs. cheetah. Go! Oh, here we go. - Go, Jimmy! Drive!\n- Oh, what is this acceleration? -

In [74]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
prompt= PromptTemplate(
    template="""
            You are a helpful AI assistant ,
            answer only from the provideded transcript ,
            if the context is insufficient , just say you dont know
            {context}
            Question: {question}
            """,
    input_variables=["context","question"]
)


In [75]:
question="Give few facts about the cheetah as mentioned ?"
context= retriever.invoke(question)
print(context)

[Document(id='42f66ec7-fa00-465c-ab72-d0c61312d087', metadata={}, page_content="more than a century ago, over 100,000 cheetahs were\nalive in the wild. But today with less than\n8,000 remaining, it's our job to protect these extraordinary\ncreatures from extinction, so that they can keep inspiring us\nall to push beyond our limits. Millie is like the sweetest animal\nI've ever met."), Document(id='295520b1-2c89-4b66-9430-e62f12b88792', metadata={}, page_content="on our set. It's so soft! The cheetah is purring\ninsanely loud. That means it\nlikes us. It looks like Millie is ready for\nthe next race. Let's get this on\nthe road. It's finally time for the moment\nwe've all been waiting for. What you're about to witness is the\nultimate showdown of man vs. animal. This race is so monumental,\nin fact, that if anyone is going to ruin it,\nit might as well be Nolan. Let's answer the most important\nquestion in modern history… - Three!\n- Can a cheetah…. - Two!\n- …outrace a Formula E car? O

In [76]:
context_text= "\n\n".join(doc.page_content for doc in context)
print(context_text)

more than a century ago, over 100,000 cheetahs were
alive in the wild. But today with less than
8,000 remaining, it's our job to protect these extraordinary
creatures from extinction, so that they can keep inspiring us
all to push beyond our limits. Millie is like the sweetest animal
I've ever met.

on our set. It's so soft! The cheetah is purring
insanely loud. That means it
likes us. It looks like Millie is ready for
the next race. Let's get this on
the road. It's finally time for the moment
we've all been waiting for. What you're about to witness is the
ultimate showdown of man vs. animal. This race is so monumental,
in fact, that if anyone is going to ruin it,
it might as well be Nolan. Let's answer the most important
question in modern history… - Three!
- Can a cheetah…. - Two!
- …outrace a Formula E car? One! Let's find out. Go! That was so fast. The car just flew past
the cheetah! Oh my God! Although Millie
didn't win, it is crazy to me that
a cheetah can hold its own against th

In [77]:
final_prompt= prompt.invoke({"context":context_text,"question":question})
print(final_prompt)

text="\n            You are a helpful AI assistant ,\n            answer only from the provideded transcript ,\n            if the context is insufficient , just say you dont know\n            more than a century ago, over 100,000 cheetahs were\nalive in the wild. But today with less than\n8,000 remaining, it's our job to protect these extraordinary\ncreatures from extinction, so that they can keep inspiring us\nall to push beyond our limits. Millie is like the sweetest animal\nI've ever met.\n\non our set. It's so soft! The cheetah is purring\ninsanely loud. That means it\nlikes us. It looks like Millie is ready for\nthe next race. Let's get this on\nthe road. It's finally time for the moment\nwe've all been waiting for. What you're about to witness is the\nultimate showdown of man vs. animal. This race is so monumental,\nin fact, that if anyone is going to ruin it,\nit might as well be Nolan. Let's answer the most important\nquestion in modern history… - Three!\n- Can a cheetah…. - T

GENERATION

In [78]:
answer=llm.invoke(final_prompt)
print(answer.content)

More than a century ago, over 100,000 cheetahs were alive in the wild.  Today, less than 8,000 remain.  A cheetah's purr is described as "insanely loud," indicating it likes the people interacting with it.  Cheetahs are described as being like "giant cats" and very soft.


CHAIN

In [60]:
from langchain_core.runnables import RunnablePassthrough,RunnableLambda,RunnableParallel
from langchain_core.output_parsers import StrOutputParser

In [84]:
def format(context):
  context_text= "\n\n".join(doc.page_content for doc in context)
  return context_text

In [85]:
parallel_chain= RunnableParallel({
    "context": retriever| RunnableLambda(format),
    "question":RunnablePassthrough()
})
parallel_chain.invoke("What is the cheetah ?")

{'context': "more than a century ago, over 100,000 cheetahs were\nalive in the wild. But today with less than\n8,000 remaining, it's our job to protect these extraordinary\ncreatures from extinction, so that they can keep inspiring us\nall to push beyond our limits. Millie is like the sweetest animal\nI've ever met.\n\non our set. It's so soft! The cheetah is purring\ninsanely loud. That means it\nlikes us. It looks like Millie is ready for\nthe next race. Let's get this on\nthe road. It's finally time for the moment\nwe've all been waiting for. What you're about to witness is the\nultimate showdown of man vs. animal. This race is so monumental,\nin fact, that if anyone is going to ruin it,\nit might as well be Nolan. Let's answer the most important\nquestion in modern history… - Three!\n- Can a cheetah…. - Two!\n- …outrace a Formula E car? One! Let's find out. Go! That was so fast. The car just flew past\nthe cheetah! Oh my God! Although Millie\ndidn't win, it is crazy to me that\na c

In [87]:
parser=StrOutputParser()


In [88]:
main_chain= parallel_chain| prompt|llm|parser

In [89]:
main_chain.invoke("summarize the video ")


'The provided text describes several unrelated events.  One segment depicts a tense physical challenge ("the circle") where a participant, Nick, struggles against an opponent, narrowly avoiding defeat. Another describes people evading dogs, using various strategies like masking their scent and seeking cover. A third segment shows elephants participating in a painting competition.  There is no single overarching summary for the entire text.'